In [5]:
from data.util.mask import bbox2mask, brush_stroke_mask, random_bbox, get_irregular_mask
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

path = "./mask/irregular/0.4-0.6/"
if not os.path.exists(path):
    os.mkdir(path)
rate = 0
for i in tqdm(range(400)):
    # regular_mask = bbox2mask((256, 256), random_bbox())
    # irregular_mask = brush_stroke_mask((256,256), )
    # mask = regular_mask | irregular_mask
    mask = get_irregular_mask((256, 256), (0.2, 0.4))
    rate += np.sum(mask) / (256*256)
    mask = mask.transpose(2,0,1)
    mask = mask.squeeze()
    # img = Image.fromarray(mask).convert("L")
    # img.save(f"{path}{i}.png")
print(rate / 400)
    # mask = np.asarray(Image.open("./mask/0.png"))
    # print(mask)
    # img = np.asarray(Image.open("./mask/irregular/100.png"))
    # plt.imshow(mask)
    # plt.show()


100%|██████████| 400/400 [00:00<00:00, 1224.38it/s]

0.3031959533691406


In [1]:

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

data_path = "/data/dataset/places365/val/"
mask_dir = "./mask/irregular/0.2-0.4/"
output = "./test_image/places2/0.2-0.4/"
if not os.path.exists(output):
    os.makedirs(output, exist_ok=True)

os.makedirs(output+"images", exist_ok=True)
os.makedirs(output+"masks", exist_ok=True)
imgs = []

for root, _, files in os.walk(data_path):
    for file in files:
        imgs.append(os.path.join(root, file))

imgs = sorted(imgs)
count = 0
for img_path in tqdm(imgs[:5000]):
    name = img_path.split("/")[-1].replace("jpg", "png")
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).transpose(2,0,1)
    mask = Image.open(mask_dir+f"{count % 400}.png")
    mask = np.array(mask)
    img = img*(1-mask) + mask * 255
    mask = mask * 255
    img = Image.fromarray(img.transpose(1,2,0))
    mask = Image.fromarray(mask)
    img.save(output+"images/"+name)
    mask.save(output+"masks/"+name)
    count += 1





100%|██████████| 5000/5000 [02:25<00:00, 34.40it/s]


In [4]:
import torch
import numpy as np
from PIL import Image

# print(np.random.normal(0, 1, (10, 10)))
# print(torch.rand((10,10)))
# print(torch.randn((10, 10)))
img = Image.open("./mask/irregular/0.1-0.2/0.png")
img = np.array(img)
rate = np.sum(img) / (512*512)
print(rate)

0.04409027099609375


In [22]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt 
from metrics.ssim import ssim
import torch
from torchvision.transforms import transforms

path = "/data/dataset/CelebA-HQ/train/female/"
imgs = []
for root, _ ,files in os.walk(path):
    for file in files:
        imgs.append(os.path.join(root, file))

target = Image.open("./1234321.jpg").resize((256,256))
tf = transforms.Compose([transforms.ToTensor(),
                         transforms.ConvertImageDtype(torch.float),
                         transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5,0.5])])
targets = tf(target)

for img_path in tqdm(imgs):
    img = Image.open(img_path).resize((256,256))
    input = tf(img)
    score = ssim(input.cuda(), targets.cuda())
    if score > 0.70:
        print(img_path.split("/")[-1])
        # break

 33%|███▎      | 5918/17943 [01:52<03:44, 53.54it/s]

066590.jpg


 73%|███████▎  | 13184/17943 [04:09<01:30, 52.87it/s]


KeyboardInterrupt: 